In [1]:
from volume_dataloader import CTScanDataModule, CTDataSet
from unet import UNet
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
from torch import sigmoid, where, cuda
import ct_utils

DATA_PATH = '/media/gaetano/DATA/DATA_NIFTI_JAWS/'

In [2]:
dataset = CTScanDataModule(DATA_PATH, batch_size=5)

model = UNet(in_channels=1,
             out_channels=4,
             n_blocks=4,
             start_filters=32,
             activation='relu',
             normalization='batch',
             conv_mode='same',
             dim=3,
             attention=False,
             loss_alpha=.5,  # .7
             loss_gamma=1, # 3/4
             learning_rate=1e-3)

lr_monitor = LearningRateMonitor(logging_interval='epoch')
checkpoint = ModelCheckpoint(monitor='val_loss')
wandb_logger = WandbLogger()
trainer = Trainer(gpus=-1,
                  log_every_n_steps=1,
                  max_epochs=50,
                  auto_lr_find=False,
                  callbacks=[checkpoint],
                  logger=wandb_logger)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
# cuda.empty_cache()
trainer.fit(model, dataset)
# checkpoint.save_checkpoint(trainer)

In [1]:
check_model_path = checkpoint.best_model_path
model.eval()
x, y = dataset.ct_val[0]
y_hat = sigmoid(model(x.unsqueeze(0)))
x = x.squeeze(0).numpy()
y_hat = y_hat.squeeze(0)
y_hat = where(y_hat > 0.5, 1, 0).numpy()
y_hat = y_hat.sum(axis=0)

NameError: name 'checkpoint' is not defined

In [2]:
%matplotlib qt
ct_utils.plot_3d_with_labels(x, y_hat, threshold=1200, transpose=[2, 1, 0], step_size=2)

NameError: name 'ct_utils' is not defined

In [3]:
# trainer.tune(model, dataset)
# Run learning rate finder
lr_finder = trainer.tuner.lr_find(model, dataset)

# Results can be found in
lr_finder.results

# Plot with
fig = lr_finder.plot(suggest=True)
fig.show()

# Pick point based on plot, or get suggestion
new_lr = lr_finder.suggestion()

# update hparams of the model
model.hparams.learning_rate = new_lr